In [1]:
!pip install advertorch

In [2]:

####################################
# imshow func
####################################
def imshow(img):
	img = img / 2 + 0.5     # unnormalize
	npimg = img.numpy()
	plt.imshow(np.transpose(npimg, (1, 2, 0)))

def _imshow(img):
	imshow(torchvision.utils.make_grid(img, normalize=True))

In [3]:

# coding: utf-8

# # Table of Contents
#  <p>

# In[1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import os
import argparse
import itertools
import torch
import torch.nn as nn

import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

from advertorch.attacks import LinfPGDAttack, CarliniWagnerL2Attack, JacobianSaliencyMapAttack, LBFGSAttack

# 								PGD Attack, 	CW Attack, 			Jacobian Attack,			FGSM Attack


# import vgg
# from vgg import VGG
from scipy.fftpack import dct, idct

from robustness.datasets import CIFAR
from robustness.model_utils import make_and_restore_model

# set whether to use GPU
torch.manual_seed(0)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D



# In[2]:


def DCT(image):
	return dct(dct(image, norm="ortho", axis=0), norm="ortho", axis=1)
def iDCT(image):
	return idct(idct(image, norm="ortho", axis=0), norm="ortho", axis=1)

class MMD_loss(nn.Module):
	def __init__(self, kernel_mul = 2.0, kernel_num = 5):
		super(MMD_loss, self).__init__()
		self.kernel_num = kernel_num
		self.kernel_mul = kernel_mul
		self.fix_sigma = None
		return
	
	def guassian_kernel(self, source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
		n_samples = int(source.size()[0])+int(target.size()[0])
		total = torch.cat([source, target], dim=0)
		total0 = total.unsqueeze(0).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
		total1 = total.unsqueeze(1).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
		L2_distance = ((total0-total1)**2).sum(2) 
		
		if fix_sigma:
			bandwidth = fix_sigma
		else:
			bandwidth = torch.sum(L2_distance.data) / (n_samples**2-n_samples)
			
		bandwidth /= kernel_mul ** (kernel_num // 2)
		bandwidth_list = [bandwidth * (kernel_mul**i) for i in range(kernel_num)]
		kernel_val = [torch.exp(-L2_distance / bandwidth_temp) for bandwidth_temp in bandwidth_list]
		return sum(kernel_val)

	def forward(self, source, target):
		batch_size = int(source.size()[0])
		kernels = self.guassian_kernel(source, target, kernel_mul=self.kernel_mul, kernel_num=self.kernel_num, fix_sigma=self.fix_sigma)
		XX = kernels[:batch_size, :batch_size]
		YY = kernels[batch_size:, batch_size:]
		XY = kernels[:batch_size, batch_size:]
		YX = kernels[batch_size:, :batch_size]
		loss = torch.mean(XX + YY - XY -YX)
		return loss
	
def lscale01(M):

	New_M = np.zeros((M.shape))
	MIN = np.min(M)
	MAX = np.max(M)
	if (MAX == MIN):
		New_M[:, :] = 0.0 * M
	else:
		New_M[:, :] = (M - MIN) / (MAX - MIN)

	return New_M


# In[3]:




'''
####################################
# model dir
####################################
if use_cuda:
	MODEL = "model.pkl"
else:
	MODEL = "model.pkl"


####################################
# load model
####################################

# Set the model 
model = vgg.vgg19()

if os.path.isfile(MODEL):
	print("=> loading model '{}'".format(MODEL))
	if use_cuda:
		model = torch.load(MODEL)['net']
		model.to(device)
	else:
		model = torch.load(MODEL, map_location=device)['net']
	print('model loaded')
else:
	print("=> no checkpoint found at '{}'".format(MODEL))
'''


# from CPU_utils import make_and_restore_model_CPU_only

# MODEL = "cifar_nat.pt"

# ds = CIFAR('data/cifar-10-batches-py')

# if use_cuda:
# 	model, _ = make_and_restore_model(arch='resnet50', dataset=ds,
# 			resume_path=MODEL, parallel=False)
# else:
# 	model, _ = make_and_restore_model_CPU_only(arch='resnet50', dataset=ds,
# 				resume_path=MODEL, parallel=False)

# model = model.model
# model.eval()



import torchvision.models as models

# resnet18 = models.resnet18(pretrained=True)
# vgg16 = models.vgg16(pretrained=True)


alexnet = models.alexnet(pretrained=True)
# squeezenet = models.squeezenet1_0(pretrained=True)
# densenet = models.densenet161(pretrained=True)
# inception = models.inception_v3(pretrained=True)
# googlenet = models.googlenet(pretrained=True)
# shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
# mobilenet = models.mobilenet_v2(pretrained=True)
# resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
# wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)
# mnasnet = models.mnasnet1_0(pretrained=True)

model = alexnet


# pick from transfer_learnt model
model = torch.load("saved_alexnet_transfer_learnt_10classes.pth")
# important before starting inferencing
model.eval()

####################################
# load data
####################################
batch_size = 200
num_folds = 0

transform=transforms.Compose([
                    transforms.Pad(4),
                    transforms.RandomCrop(96),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                ])

testset = torchvision.datasets.STL10(root='../data',download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,shuffle=False, num_workers=0)

classes =  ('airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck')

# In[4]:


####################################
# Construct an adversary instance
####################################
adversary_CW = CarliniWagnerL2Attack(model, num_classes = len(classes) , 
	confidence=0, targeted=False, learning_rate=0.01, binary_search_steps=9, max_iterations=10000, 
	abort_early=True, initial_const=0.001, clip_min=0.0, clip_max=1.0, loss_fn=None)

adversary_Jacobian = JacobianSaliencyMapAttack(model, num_classes = len(classes), 
	clip_min=0.0, clip_max=1.0, loss_fn=None, theta=1.0, gamma=1.0, comply_cleverhans=False)

adversary_PGD = LinfPGDAttack(
	model, loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=0.15,
	nb_iter=40, eps_iter=0.01, rand_init=True, clip_min=0.0, clip_max=1.0,
	targeted=False)

adversary_FGSM = LBFGSAttack(model, num_classes = len(classes), batch_size=1, binary_search_steps=9, 
	max_iterations=100, initial_const=0.01, clip_min=0, clip_max=1, loss_fn=None, targeted=False)


# In[22]:


# Set the type of attack

loss = MMD_loss()


# adversary = adversary_CW
# FOLDER = 'adversary_CW/'



adversary = adversary_PGD
FOLDER = './adversary_PGD/'




filetxt = FOLDER+'output.txt'
if not os.path.exists(FOLDER):
	os.makedirs(FOLDER)

true = np.zeros((0)).astype(int)
pred_cln = np.zeros((0)).astype(int)
pred_untargeted_adv = np.zeros((0)).astype(int)
pred_targeted_adv = np.zeros((0)).astype(int)

MMD_untargeted = np.zeros((0))
MMD_targeted = np.zeros((0))


fold = 0
DCT_untargeted = 0
DCT_targeted = 0
	

Files already downloaded and verified


In [4]:
cln_data, true_label = next(iter(testloader))

_, pred_cln_ = torch.max(model(cln_data), 1)

cln_data

pred_cln_

tensor([4, 5, 4, 6, 4, 9, 4, 4, 5, 8, 0, 6, 5, 8, 7, 6, 6, 6, 2, 2, 4, 1, 6, 2,
        1, 8, 6, 1, 5, 7, 0, 0, 2, 1, 3, 6, 5, 2, 5, 7, 0, 6, 1, 3, 7, 4, 3, 0,
        6, 7, 6, 5, 8, 0, 6, 1, 6, 6, 8, 2, 2, 2, 9, 5, 1, 7, 8, 5, 8, 8, 9, 0,
        5, 2, 2, 1, 6, 7, 8, 9, 5, 0, 8, 3, 2, 6, 6, 6, 8, 3, 5, 9, 6, 6, 6, 7,
        0, 5, 0, 5, 2, 7, 1, 0, 0, 8, 0, 4, 2, 4, 3, 2, 5, 5, 1, 2, 6, 6, 2, 2,
        8, 9, 5, 7, 0, 5, 1, 3, 6, 3, 6, 4, 8, 5, 1, 3, 7, 4, 6, 2, 9, 5, 2, 2,
        2, 8, 8, 6, 1, 6, 8, 2, 0, 3, 2, 3, 5, 7, 8, 7, 2, 2, 7, 8, 9, 8, 2, 7,
        4, 3, 5, 6, 7, 7, 6, 9, 1, 2, 2, 7, 9, 4, 1, 0, 4, 4, 2, 4, 4, 6, 5, 3,
        4, 1, 3, 5, 4, 2, 0, 7])

In [5]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [13]:



if use_cuda:
	cln_data = cln_data.cuda()
	true_label = true_label.cuda()
	model = model.cuda()
	

for cln_data, true_label in testloader:

	print("working for fold number :", fold,"\n")

	cln_data, true_label = cln_data.to(device), true_label.to(device)   
	model = model.to(device)
	true = np.concatenate((true, true_label.cpu().numpy().astype(int)), axis=None)
	
	_, pred_cln_ = torch.max(model(cln_data), 1)
	pred_cln = np.concatenate((pred_cln, pred_cln_.cpu().numpy().astype(int)), axis=None)

	# select only good samples for showcasing
	compare_prediction_label = (pred_cln == true)
	good_samples = [i for i, x in enumerate(compare_prediction_label) if x == True]    
	
	####################################
	#Perform untargeted attack	
	####################################
	
	print("starting untargeted attack")
	adv_untargeted = adversary.perturb(cln_data, true_label)
	print("Completed untargeted attack !")


	_, pred_untargeted_adv_ = torch.max(model(adv_untargeted), 1)
	pred_untargeted_adv = np.concatenate((pred_untargeted_adv, pred_untargeted_adv_.cpu().numpy().astype(int)), axis=None)
	
	####################################
	# perform targeted attack
	####################################
	target = torch.ones_like(true_label) * 3
	adversary.targeted = True
	

	print("starting targeted attack ...")
	adv_targeted = adversary.perturb(cln_data, target)
	print("Completed untargeted attack !")
	
	_, pred_adv_targeted_ = torch.max(model(adv_targeted), 1)
	pred_targeted_adv = np.concatenate((pred_targeted_adv, pred_adv_targeted_.cpu().numpy().astype(int)), axis=None)

	for i in range(len(true_label)):
		image = np.transpose(adv_untargeted[i].cpu().numpy(), (1, 2, 0))
		dct_adv_untargeted = ((DCT(image[:,:,0]) + DCT(image[:,:,1]) + DCT(image[:,:,2]))/3.0)

		image = np.transpose(adv_targeted[i].cpu().numpy(), (1, 2, 0))
		dct_adv_targeted = ((DCT(image[:,:,0]) + DCT(image[:,:,1]) + DCT(image[:,:,2]))/3.0)

		image = np.transpose(cln_data[i].cpu().numpy(), (1, 2, 0))
		dct_image = ((DCT(image[:,:,0]) + DCT(image[:,:,1]) + DCT(image[:,:,2]))/3.0)

		DCT_untargeted = DCT_untargeted + abs((dct_image - dct_adv_untargeted)/dct_image)
		DCT_targeted = DCT_targeted + abs((dct_image - dct_adv_targeted)/dct_image)

		MMD_targeted = np.append(MMD_targeted,loss(torch.from_numpy(dct_adv_targeted), torch.from_numpy(dct_image)).cpu().item())
		MMD_untargeted = np.append(MMD_untargeted,loss(torch.from_numpy(dct_adv_untargeted), torch.from_numpy(dct_image)).cpu().item())
	fold += 1
	if(fold > num_folds):
		break
	

working for fold number : 1 

starting untargeted attack
Completed untargeted attack !
starting targeted attack ...
Completed untargeted attack !


In [ ]:
	
####################################
# Visualization of attacks
####################################
import matplotlib.pyplot as plt

acc_adv_untargeted = 100*np.mean(true == pred_untargeted_adv)
acc_adv_targeted = 100*np.mean(true == pred_targeted_adv)
acc_cln = 100*np.mean(true == pred_cln)

print('Total Accuracy: ',"Unattacked: ", acc_cln, 
	  '\t and during attack: ',FOLDER,'\t',
	  
	  "Untargeted attack: ",acc_adv_untargeted,
	  "\tTargeted attack: ",acc_adv_targeted,
	 file=open(filetxt, "a"))

correct_pred_indx = (true == pred_cln)
acc_adv_untargeted = 100*np.mean(true[correct_pred_indx] == pred_untargeted_adv[correct_pred_indx])
acc_adv_targeted = 100*np.mean(true[correct_pred_indx] == pred_targeted_adv[correct_pred_indx])
print('Effect of attack on accurately predcited image by unattacked model:\t'      
	  "Untargeted attack: ",acc_adv_untargeted,
	  "\tTargeted attack: ",acc_adv_targeted,
	 file=open(filetxt, "a"))

DCT_untargeted = (DCT_untargeted/len(true))
DCT_targeted = (DCT_targeted/len(true))

fig, ax = plt.subplots(1,1,figsize=(6, 6))
im1 = ax.imshow(lscale01(DCT_untargeted), cmap='YlOrRd')
ax.title.set_text('untargeted')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)    
fig.colorbar(im1, cax=cax)
# ax.set_axis_off()
plt.savefig(FOLDER+'correct_DCT_untargeted.png')
# plt.show()

fig, ax = plt.subplots(1,1,figsize=(6, 6))
im1 = ax.imshow(lscale01(DCT_targeted), cmap='YlOrRd')
ax.title.set_text('targeted')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)    
fig.colorbar(im1, cax=cax)
# ax.set_axis_off()
plt.savefig(FOLDER+'correct_DCT_targeted.png')
# plt.show()


c_indx_untargeted = np.logical_and(true == pred_cln,true == pred_untargeted_adv)
c_indx_targeted = np.logical_and(true == pred_cln,true == pred_targeted_adv)

true_pred_untargeted = np.sum(c_indx_untargeted)/np.sum(correct_pred_indx)
true_pred_targeted = np.sum(c_indx_targeted)/np.sum(correct_pred_indx)

w_indx_untargeted = np.logical_and(true == pred_cln,true != pred_untargeted_adv)
w_indx_targeted = np.logical_and(true == pred_cln,true != pred_targeted_adv)

false_pred_untargeted = np.sum(w_indx_untargeted)/np.sum(correct_pred_indx)
false_pred_targeted = np.sum(w_indx_targeted)/np.sum(correct_pred_indx)

print("True pred accuracy of samples: ",100*true_pred_untargeted,100*true_pred_targeted,
	 file=open(filetxt, "a"))
print("wrong pred accuracy of samples: ",100*false_pred_untargeted,100*false_pred_targeted,
	 file=open(filetxt, "a"))

print("Total mean MMD: ", np.mean(MMD_untargeted), 
	  np.mean(MMD_targeted),
	 file=open(filetxt, "a"))
print("Correct mean MMD: ", np.mean(MMD_untargeted[c_indx_untargeted]), 
	  np.mean(MMD_targeted[c_indx_targeted]),
	 file=open(filetxt, "a"))
print("wrong mean MMD: ", np.mean(MMD_untargeted[w_indx_untargeted == 0]), 
	  np.mean(MMD_targeted[w_indx_targeted == 0]),
	 file=open(filetxt, "a"))

fig, ax = plt.subplots(1,1,figsize=(6, 6))
plt.hist(MMD_untargeted,bins=100, weights=100*np.ones(len(MMD_untargeted)) / len(MMD_untargeted))
plt.xlabel("MMD_untargeted")
plt.ylabel("% of images")
plt.title('Histogram of MMD_untargeted')
plt.axvline(np.mean(MMD_untargeted), color='k', linestyle='dashed', linewidth=1)
plt.savefig(FOLDER+'hist_untargeted.png')
# plt.show()

fig, ax = plt.subplots(1,1,figsize=(6, 6))
plt.hist(MMD_targeted,bins=100, weights=100*np.ones(len(MMD_targeted)) / len(MMD_targeted))
plt.xlabel("MMD_targeted")
plt.ylabel("% of images")
plt.title('Histogram of MMD_targeted')
plt.axvline(np.mean(MMD_untargeted), color='k', linestyle='dashed', linewidth=1)
plt.savefig(FOLDER+'hist_targeted.png')
# plt.show()


# In[183]:


# if use_cuda:
# 	cln_data = cln_data.cpu()
# 	true_label = true_label.cpu()
# 	adv_untargeted = adv_untargeted.cpu()
# 	adv_targeted = adv_targeted.cpu()
	


# %%


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/usr/local/lib/python3

In [49]:
####################################
# imshow func
####################################
def imshow(img):
  img = img.cpu()
  img = img / 2 + 0.5     # unnormalize
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg, (1, 2, 0)))

def _imshow(img):
  img = img / 2 + 0.5     # unnormalize
	# imshow(torchvision.utils.make_grid(img, normalize=True))

  plt.imshow(torchvision.utils.make_grid(
        img, normalize=True).numpy().transpose((1, 2, 0)))

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-49-3850063ba5d1>, line 12)

In [50]:
for ii in range(num_plots):
    
    print(pred_cln[ii])

_imshow(cln_data[0])

3
6
4
6
4
9
4
4


In [51]:

plt.figure(figsize=(10, 8))
num_plots = 8
#print(classes, pred_cln[ii])
for jj in range(num_plots):

	ii = good_samples[jj]

	plt.subplot(3, num_plots, jj + 1)
	_imshow(cln_data[ii])
	plt.title("clean \n pred: {}".format(classes[pred_cln[ii]]))
	plt.subplot(3, num_plots, jj + 1 + num_plots)
	_imshow(adv_untargeted[ii])
	plt.title("untargeted \n adv \n pred: {}".format(classes[pred_untargeted_adv[ii]]))
	plt.subplot(3, num_plots, jj + 1 + num_plots * 2)
	_imshow(adv_targeted[ii])
	plt.title("target cat \n adv \n pred: {}".format(classes[pred_targeted_adv[ii]]))

plt.tight_layout()
plt.savefig(FOLDER+'test.png')
